In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import io
import base64
import numpy as np
import pandas as pd
import seaborn as sns
from typing import Dict, Any
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('synthetic_fraud_dataset.csv')

In [3]:
df.shape

(1000, 12)

In [4]:
df.columns

Index(['transaction_id', 'datetime', 'transaction_amount', 'transaction_time',
       'account_age_days', 'num_prev_transactions', 'avg_transaction_amount',
       'transaction_type', 'device_type', 'region', 'customer_segment',
       'is_fraud'],
      dtype='object')

In [5]:
df.head()

,transaction_id,datetime,transaction_amount,transaction_time,account_age_days,num_prev_transactions,avg_transaction_amount,transaction_type,device_type,region,customer_segment,is_fraud
0,1,2025-01-16 07:27:17.813857,205.18,17.508197,399,26,88.69,payment,mobile,South,individual,0
1,2,2025-01-23 05:41:17.813857,192.78,18.372310,1009,16,52.97,payment,mobile,East,individual,0
2,3,2025-07-04 16:07:17.813857,142.37,3.813796,534,23,98.79,payment,mobile,East,business,0
3,4,2025-04-21 14:27:17.813857,38.12,14.645404,1273,24,92.05,transfer,mobile,North,student,0
4,5,2025-01-08 04:37:17.813857,107.06,3.248498,1865,18,141.76,purchase,mobile,West,individual,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   transaction_id          1000 non-null   int64  
 1   datetime                1000 non-null   object 
 2   transaction_amount      1000 non-null   float64
 3   transaction_time        1000 non-null   float64
 4   account_age_days        1000 non-null   int64  
 5   num_prev_transactions   1000 non-null   int64  
 6   avg_transaction_amount  1000 non-null   float64
 7   transaction_type        1000 non-null   object 
 8   device_type             1000 non-null   object 
 9   region                  1000 non-null   object 
 10  customer_segment        1000 non-null   object 
 11  is_fraud                1000 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 93.9+ KB


In [7]:
df.describe()

,transaction_id,transaction_amount,transaction_time,account_age_days,num_prev_transactions,avg_transaction_amount,is_fraud
count,1000.000000,1000.00000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,500.500000,124.50263,12.051881,991.55100,19.742000,81.432500,0.280000
std,288.819436,125.75085,6.983157,565.95852,4.387789,30.530074,0.449224
min,1.000000,0.39000,0.000279,12.00000,9.000000,1.000000,0.000000
25%,250.750000,33.42500,6.246110,512.25000,17.000000,60.762500,0.000000
50%,500.500000,87.76000,12.044499,959.50000,20.000000,80.665000,0.000000
75%,750.250000,171.77000,18.184241,1476.25000,23.000000,101.770000,1.000000
max,1000.000000,893.01000,23.947701,1998.00000,34.000000,201.580000,1.000000


In [8]:
df.isnull().sum()

transaction_id            0
datetime                  0
transaction_amount        0
transaction_time          0
account_age_days          0
num_prev_transactions     0
avg_transaction_amount    0
transaction_type          0
device_type               0
region                    0
customer_segment          0
is_fraud                  0
dtype: int64

In [9]:
import numpy as np
import pandas as pd
from typing import Dict, Any
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

In [10]:
def _compute_overview_and_stats(df: pd.DataFrame) -> Dict[str, Any]:
    overview = {
        'total_rows': int(df.shape[0]),
        'total_columns': int(df.shape[1]),
        'missing_values': int(df.isna().sum().sum()),
        'numeric_columns': int(df.select_dtypes(include=[np.number]).shape[1]),
    }

    column_info = []
    for col in df.columns:
        series = df[col]
        missing_percent = float(series.isna().mean() * 100.0)
        dtype_str = str(series.dtype)
        unique_values = int(series.nunique(dropna=True))
        column_info.append({
            'name': col,
            'dtype': dtype_str,
            'missing_percent': missing_percent,
            'unique_values': unique_values,
        })

    numeric_df = df.select_dtypes(include=[np.number])
    stats: Dict[str, Dict[str, float]] = {}
    if not numeric_df.empty:
        desc = numeric_df.describe().to_dict()
        # transpose to {col: {metric: value}}
        for col, metrics in desc.items():
            stats[col] = {}
            for metric, value in metrics.items():
                # Cast numpy types to python native for JSON
                if pd.isna(value):
                    stats[col][metric] = None
                else:
                    stats[col][metric] = float(value)

    return {
        'data_overview': overview,
        'column_info': column_info,
        'statistics': stats,
    }

def _apply_preprocessing(df: pd.DataFrame, steps: list[str]) -> pd.DataFrame:
    result = result.copy(deep=True)
    numeric_cols = list(result.select_dtypes(include=[np.number]).columns)
    cat_cols = list(result.select_dtypes(exclude=[np.number]).columns)

    if 'drop_missing' in steps:
        result = result.dropna()
        numeric_cols = list(result.select_dtypes(include=[np.number]).columns)
        cat_cols = list(result.select_dtypes(exclude=[np.number]).columns)

    if 'fill_mean' in steps and numeric_cols:
        result[numeric_cols] = result[numeric_cols].fillna(result[numeric_cols].mean())

    if 'fill_median' in steps and numeric_cols:
        result[numeric_cols] = result[numeric_cols].fillna(result[numeric_cols].median())

    if 'fill_mode' in steps and cat_cols:
        for c in cat_cols:
            mode_val = result[c].mode(dropna=True)
            if not mode_val.empty:
                result[c] = result[c].fillna(mode_val.iloc[0])

    # One-hot encoding
    if 'one_hot' in steps and cat_cols:
        result = pd.get_dummies(result, columns=cat_cols, drop_first=True)
        numeric_cols = list(result.select_dtypes(include=[np.number]).columns)

    ## Outliear Treatment using IQR Mapping
    if 'treat_outliers' in steps and numeric_cols:
        for col in numeric_cols:
            Q1 = result[col].quantile(0.25)
            Q3 = result[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            result[col] = np.where(result[col] < lower_bound, lower_bound,
                                   np.where(result[col] > upper_bound, upper_bound, result[col]))

    # Scaling / Normalization (apply to numeric columns only)
    numeric_cols = list(result.select_dtypes(include=[np.number]).columns)
    if numeric_cols:
        if 'standardize' in steps:
            scaler = StandardScaler()
            result[numeric_cols] = scaler.fit_transform(result[numeric_cols])
        if 'minmax' in steps:
            scaler = MinMaxScaler()
            result[numeric_cols] = scaler.fit_transform(result[numeric_cols])
        if 'robust' in steps:
            scaler = RobustScaler()
            result[numeric_cols] = scaler.fit_transform(result[numeric_cols])
        if 'normalize_l2' in steps:
            normalizer = Normalizer(norm='l2')
            result[numeric_cols] = normalizer.fit_transform(result[numeric_cols])

    return result

In [14]:
def _figure_to_base64() -> str:
    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close()
    buf.seek(0)
    return base64.b64encode(buf.read()).decode('utf-8')

def _generate_plots(df: pd.DataFrame) -> Dict[str, Any]:
    plots: Dict[str, Any] = {}
    numeric_df = df.select_dtypes(include=[np.number])

    # Correlation heatmap
    if numeric_df.shape[1] >= 2:
        plt.figure(figsize=(6, 5))
        corr = numeric_df.corr(numeric_only=True)
        sns.heatmap(corr, cmap='coolwarm', annot=False)
        plots['heatmap'] = _figure_to_base64()

    # Histograms and Boxplots for up to 6 numeric columns
    cols = list(numeric_df.columns)[:6]

    if cols:
        ## Historgram subplot
        n_cols = 2 # 2 plots per row
        n_rows = (len(cols) + n_cols - 1) // n_cols
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 3))
        axes = axes.flatten()
        for i, col in enumerate(cols):
            series = numeric_df[col].dropna()
            if series.empty:
                axes[i].axis('off')
                continue
            sns.histplot(series, kde=True, ax=axes[i], color='#6366F1')
            axes[i].set_title(f'Histogram of {col}')
        ## Turn off any unused subplots
        for j in range(i + 1, len(axes)):
            axes[j].axis('off')
        plots['histograms'] = _figure_to_base64()

        ## Boxplot subplot
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 2.5))
        axes = axes.flatten()
        for i, col in enumerate(cols):
            series = numeric_df[col].dropna()
            if series.empty:
                axes[i].axis('off')
                continue
            sns.boxplot(x=series, ax=axes[i], color='#22C55E')
            axes[i].set_title(f'Boxplot of {col}')
        ## Turn off any unused subplots
        for j in range(i + 1, len(axes)):
            axes[j].axis('off')
        plots['boxplots'] = _figure_to_base64()

    return plots

In [15]:
_generate_plots(df)

{'heatmap': '...',
 'histograms': '...',
 'boxplots': '...'}